In [1]:
import spacy
import en_core_web_lg
from spacy import displacy
import collections

nlp = en_core_web_lg.load()
print(type(nlp))

<class 'spacy.lang.en.English'>


In [2]:
import json

with open('../scraping/mike/output2.json', 'r') as f:
    docs = json.loads(f.read())

In [3]:
from bs4 import BeautifulSoup
from bs4.element import Comment

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.find(id='bodyContent').findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.replace('\xa0', ' ').strip() for t in visible_texts)

In [4]:
%%time
entities = collections.Counter()
for doc in nlp.pipe([text_from_html(doc['html']) for doc in docs], batch_size=50):
    entities.update([ent.text for ent in doc.ents])

Wall time: 19min 39s


In [5]:
len(entities)

180482

In [6]:
import pandas as pd
elist = list(entities.items())
entity_df = pd.DataFrame(data={'term': [e[0] for e in elist], 'count': [e[1] for e in elist]})
entity_df.head()

,term,count
0,LinkedIn Corporation Initial,3
1,May 2010,104
2,9 years ago,13
3,2010-05,3
4,Platform Web Browser,4


In [8]:
entity_df.to_csv('entity_list.tsv', sep='\t', index=False)